In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Let's load the data

In [ ]:
import csv
csv_filename = "/kaggle/input/clash-royale-season-18-dec-0320-dataset/BattlesStaging_12272020_WL_tagged/battlesStaging_12272020_WL_tagged.csv"
chunk_size = 500000

text_file_reader = pd.read_csv(csv_filename, engine='python',encoding='utf-8-sig', quoting=csv.QUOTE_MINIMAL, chunksize = chunk_size,index_col=0)

### Stitch the indivdual dataframes as one

In [ ]:
dfList = []
counter = 0

for df in text_file_reader:
    dfList.append(df)
    counter= counter +1
    print("Max rows read: " + str(chunk_size * counter) )

df = pd.concat(dfList,sort=False)

### Load the win-conditions, this is based here https://www.deckshop.pro/card/flag/win-condition

In [ ]:
wincon_csv_filename = "/kaggle/input/clash-royale-season-18-dec-0320-dataset/Wincons.csv"
df_wincon = pd.read_csv(wincon_csv_filename, engine='python',encoding='utf-8-sig', quoting=csv.QUOTE_MINIMAL, index_col = 0)
wincon_dict = dict(zip(df_wincon["card_id"],df_wincon["card_name"]))

### Define Arena Placer Code, average starting trophies of 2 players will be used

In [ ]:
def arena_placer(row):
    arena_name = None
    
    if(row['average.startingTrophies'] < 300):
        arena_name = '01.Goblin Stadium'
    elif ((row['average.startingTrophies'] >= 300)&(row['average.startingTrophies'] < 600)):
        arena_name = '02.Bone Pit' 
    elif ((row['average.startingTrophies'] >= 600)&(row['average.startingTrophies'] < 1000)):
        arena_name = '03.Barbarian Bowl'
    #1000
    elif ((row['average.startingTrophies'] >= 1000)&(row['average.startingTrophies'] < 1300)):
        arena_name = '04.P.E.K.K.A\'s Playhouse'
    elif ((row['average.startingTrophies'] >= 1300)&(row['average.startingTrophies'] < 1600)):
        arena_name = '05.Spell Valley'  
    elif ((row['average.startingTrophies'] >= 1600)&(row['average.startingTrophies'] < 2000)):
        arena_name = '06.Builder\'s Workshop'
    # 2000
    elif ((row['average.startingTrophies'] >= 2000)&(row['average.startingTrophies'] < 2300)):
        arena_name = '07.Royal Arena'
    elif ((row['average.startingTrophies'] >= 2300)&(row['average.startingTrophies'] < 2600)):
        arena_name = '08.Frozen Peak'  
    elif ((row['average.startingTrophies'] >= 2600)&(row['average.startingTrophies'] < 3000)):
        arena_name = '09.Jungle Arena'
    # 3000
    elif ((row['average.startingTrophies'] >= 3000)&(row['average.startingTrophies'] < 3300)):
        arena_name = '10.Hog Mountain'
    elif ((row['average.startingTrophies'] >= 3300)&(row['average.startingTrophies'] < 3600)):
        arena_name = '11.Electro Valley'  
    elif ((row['average.startingTrophies'] >= 3600)&(row['average.startingTrophies'] < 4000)):
        arena_name = '12.Spooky Town'
    # 4000
    elif ((row['average.startingTrophies'] >= 4000)&(row['average.startingTrophies'] < 4300)):
        arena_name = '13.Challenger I'
    elif ((row['average.startingTrophies'] >= 4300)&(row['average.startingTrophies'] < 4600)):
        arena_name = '14.Challenger II'
    elif ((row['average.startingTrophies'] >= 4600)&(row['average.startingTrophies'] < 5000)):
        arena_name = '15.Challenger III'
    # 5000
    elif ((row['average.startingTrophies'] >= 5000)&(row['average.startingTrophies'] < 5300)):
        arena_name = '16.Master I'
    elif ((row['average.startingTrophies'] >= 5300)&(row['average.startingTrophies'] < 5600)):
        arena_name = '17.Master II'
    elif ((row['average.startingTrophies'] >= 5600)&(row['average.startingTrophies'] < 6000)):
        arena_name = '18.Master III'
    # 6000
    elif ((row['average.startingTrophies'] >= 6000)&(row['average.startingTrophies'] < 6300)):
        arena_name = '19.Champion'
    elif ((row['average.startingTrophies'] >= 6300)&(row['average.startingTrophies'] < 6600)):
        arena_name = '20.Grand Champion'
    elif ((row['average.startingTrophies'] >= 6600)&(row['average.startingTrophies'] < 7000)):
        arena_name = '21.Royal Champion'         
    elif(row['average.startingTrophies'] >= 7000):
        arena_name = '22.Ultimate Champion' 
        
    return arena_name

In [ ]:
df['average.startingTrophies'] = df['average.startingTrophies'].fillna(0)
df['arena.name'] = None
df['arena.name'] = df.apply(arena_placer, axis=1)

### Define the Win Con Populator of the Deck

In [ ]:
import json
def win_con_populator(row):

    winner_win_con_id = 99999999
    loser_win_con_id = 99999999
    
    
    for key, value in wincon_dict.items():
        if((key in json.loads(row['winner.cards.list']))&(winner_win_con_id == 99999999)):
            winner_win_con_id = key
    
        if((key in json.loads(row['loser.cards.list']))&(loser_win_con_id == 99999999)):
            loser_win_con_id = key
            
        if((winner_win_con_id !=99999999)&(loser_win_con_id !=99999999)):
            break
        
    return winner_win_con_id, wincon_dict[winner_win_con_id], loser_win_con_id, wincon_dict[loser_win_con_id]

### Just a simple eye-check

In [ ]:
df.sample(n=5)

### Let's populate the win-cons of the decks

In [ ]:
df['winner.wincon.card.id'] = 99999999
df['loser.wincon.card.id'] = 99999999
df['winner.wincon.card.name'] = None
df['loser.wincon.card.name'] = None
df['winner.wincon.card.id'],df['winner.wincon.card.name'],df['loser.wincon.card.id'],df['loser.wincon.card.name'] = zip(*df.apply(win_con_populator, axis=1))

In [ ]:
df.reset_index(inplace=True)

### What columns do we have?

In [ ]:
df.columns

### Let's the get winningest/ most-winning win-con

In [ ]:
df_arena_win_counts = df.groupby(['arena.name', 'winner.wincon.card.name']).agg({"index":["count"]})
df_arena_win_counts.columns = df_arena_win_counts.columns.map('_'.join)
df_arena_win_counts = df_arena_win_counts.reset_index()
df_arena_win_counts.columns = [col.strip() for col in df_arena_win_counts.columns]
df_arena_win_counts.sort_values(["arena.name","index_count"], ascending=[True, False], inplace=True)
df_arena_win_counts['arena.name'] = df_arena_win_counts['arena.name'].str.split('.').str[1]

### Let's see what's the most winning win-con in the Ultimate Champion Arena

In [ ]:
df_arena_win_counts[df_arena_win_counts["arena.name"] == "Ultimate Champion"]

### Let's see what's the most winning win-con in the Master I Arena

In [ ]:
df_arena_win_counts[df_arena_win_counts["arena.name"] == "Master I"]

### Let's pivot the Loser WinCon column, using the Winner Arena and Winner WinCon as index/ pivoting columns

In [ ]:
df_arena_win_lose_counts = df.groupby(['arena.name', 'winner.wincon.card.name','loser.wincon.card.name']).agg({"index":["count"]})
df_arena_win_lose_counts.columns = df_arena_win_lose_counts.columns.map('_'.join)
df_arena_win_lose_counts = df_arena_win_lose_counts.reset_index()
df_arena_win_lose_counts.columns = [col.strip() for col in df_arena_win_lose_counts.columns] 

In [ ]:
df_arena_win_lose_counts_pivoted =  df_arena_win_lose_counts.pivot_table(columns='loser.wincon.card.name', index=['arena.name', 'winner.wincon.card.name'], aggfunc=sum, fill_value=0)

In [ ]:
df_arena_win_lose_counts_pivoted.columns = df_arena_win_lose_counts_pivoted.columns.map('_'.join)
df_arena_win_lose_counts_pivoted = df_arena_win_lose_counts_pivoted.reset_index()

### Let's just remove some unwanted string(s) in the row values of arena.name

In [ ]:
df_arena_win_lose_counts_pivoted['arena.name'] = df_arena_win_lose_counts_pivoted['arena.name'].str.split('.').str[1]

### Let's remove some unwanted strings in the columns

In [ ]:
df_arena_win_lose_counts_pivoted.columns = [col.replace("index_count_","") for col in df_arena_win_lose_counts_pivoted.columns] 

### Some eye-checks

In [ ]:
df_arena_win_lose_counts_pivoted

### TODO: graphs